# Term Paper Project: EU Transparency Register

Research Question: Which lobby groups have been in contanct with members of the European Commission (EC) or the Directorates-Generale (DG) ?

In [28]:
# load packages
import pandas as pd
import numpy as np
import os

# Load & Transform Data
- Load Data from Transparency register
- filter for Meetings within the relevant time period

In [22]:
# load data

meetings_ec = pd.read_excel("./data/Meetings of Commission representatives of the Von der Leyen Commission (2019-2024).xlsx", header=1)
meetings_dg = pd.read_excel("./data/Meetings of Directors-General of the European Commission.xlsx", header=1)

# transform subject column to lower case to improve keyword search
meetings_ec["Subject of the meeting"] = meetings_ec["Subject of the meeting"].str.lower()
meetings_dg["Subject of the meeting"] = meetings_dg["Subject of the meeting"].str.lower()

# filter for entries between 
min_date = "2020-06-01"
max_date = "2023-04-18"

meetings_dg['Date of meeting'] = pd.to_datetime(meetings_dg['Date of meeting'], format="%Y-%m-%d")
meetings_ec['Date of meeting'] = pd.to_datetime(meetings_ec['Date of meeting'], format="%Y-%m-%d")


meetings_dg = meetings_dg.loc[(meetings_dg['Date of meeting'] >= min_date)
                     & (meetings_dg['Date of meeting'] <= max_date)]

meetings_ec = meetings_ec.loc[(meetings_ec['Date of meeting'] >= min_date)
                     & (meetings_ec['Date of meeting'] <= max_date)]



In [ ]:
# select all relevant meetings by filtering for keywords in the column "Subject of the meeting"

chips_act_keywords = ["chips", "semiconductor", "chip", "2023/1781"]
keywords_to_exclude = ["schiphol"]

chips_meetings_ec = meetings_ec[
    (meetings_ec["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True))  
    & (meetings_ec["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
]
chips_meetings_dg = meetings_dg[
    (meetings_dg["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True)) 
    & (meetings_dg["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
    ]




In [45]:
transparency_register = pd.DataFrame(columns=["Transparency register ID", "Head Office"])

# Head office country, Identification number:


for entry in os.scandir("./data"):  
    if entry.is_file() and entry.name.endswith(".xls"):  # check if it's a xlsx file
        print(entry.path)
        xls = pd.read_excel(entry.path)

        if "Identification number:" in xls.columns:
            xls = xls[["Head office country", "Identification number:", "(Organisation) name"]] 
            xls.rename(columns={"Identification number:": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification number" in xls.columns:
            xls = xls[["Head office country", "Identification number", "(Organisation) name"]]
            xls.rename(columns={"Identification number": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification code" in xls.columns:
            xls = xls[["Head office country", "Identification code", "(Organisation) name"]]
            xls.rename(columns={"Identification code": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        else:
            raise KeyError("ERROR MISSING TRANSPARENCY REGISTER ID COLUMN!")
        transparency_register = pd.concat([transparency_register, xls]).drop_duplicates(subset="Transparency register ID")

transparency_register["Head Office"] = transparency_register["Head Office"].str.upper()
transparency_register.to_excel("./data/merged_register.xlsx", index=False)



./data\Organisations in Transparency Register - 2019 - JANUARY.xls
./data\Organisations in Transparency Register - 2019 - JUNE.xls
./data\Organisations in Transparency Register - 2020 - JANUARY.xls
./data\Organisations in Transparency Register - 2020 - JUNE.xls
./data\Organisations in Transparency Register - 2021 - JANUARY.xls
./data\Organisations in Transparency Register - 2021 - JUNE.xls
./data\Organisations in Transparency Register - 2022 - JANUARY.xls
./data\Organisations in Transparency Register - 2022 - JUNE.xls
./data\Organisations in Transparency Register - 2023 - JANUARY.xls
./data\Organisations in Transparency Register - 2023 - JUNE.xls
./data\Organisations in Transparency Register - 2024 - JAN.xls


In [ ]:
# TODO: Handle Meetings with multiple entries in Register ID Column: 

chips_meetings_dg_merged = pd.merge(left=chips_meetings_dg, right=transparency_register, 
                                    how="left", on="Transparency register ID")


